In [1]:
import pandas as pd
import numpy as np

In [2]:
data_path = '../data/full_dataset/'
train_df = pd.read_csv(data_path + 'goemotions_1.csv')
train_df.head()

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,1.548084e+09,37,True,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1.546428e+09,37,False,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1.547965e+09,18,False,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1.546669e+09,2,False,0,...,0,0,0,0,0,0,0,0,0,1


In [3]:
print(train_df.shape)
cols = train_df.columns
print(cols)

(70000, 37)
Index(['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id', 'example_very_unclear', 'admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral'],
      dtype='object')


In [4]:
unclear = train_df[train_df['example_very_unclear']==True]
print(unclear.shape)

(1129, 37)


In [5]:
pos_labels = ['admiration','approval', 'amusement', 'caring', 'desire', 
              'excitement', 'gratitude', 'joy', 'love','optimism', 'pride', 'relief']
neg_labels = ['anger', 'annoyance', 'disappointment', 'disapproval', 'disgust',
              'embarrassment','fear', 'grief', 'nervousness', 'remorse', 'sadness']
ambi_labels = ['confusion', 'curiosity', 'realization', 'surprise']

In [6]:
emotion_count = {}
category_count = {}

pos_sum = 0
for label in pos_labels:
    df = train_df[train_df[label]==1]
    emotion_count[label] = len(df)
    pos_sum += len(df)
category_count['positive'] = pos_sum

neg_sum = 0
for label in neg_labels:
    df = train_df[train_df[label]==1]
    emotion_count[label] = len(df)
    neg_sum += len(df)
category_count['negative'] = neg_sum

ambi_sum = 0
for label in ambi_labels:
    df = train_df[train_df[label]==1]
    emotion_count[label] = len(df)
    ambi_sum += len(df)

df = train_df[train_df['neutral']==1]
emotion_count['neutral'] = len(df)
ambi_sum += len(df)
category_count['neutral'] = ambi_sum

In [7]:
emotion_count

{'admiration': 5647,
 'approval': 5928,
 'amusement': 3081,
 'caring': 1988,
 'desire': 1248,
 'excitement': 1900,
 'gratitude': 3863,
 'joy': 2607,
 'love': 2745,
 'optimism': 2887,
 'pride': 452,
 'relief': 452,
 'anger': 2589,
 'annoyance': 4443,
 'disappointment': 2771,
 'disapproval': 3774,
 'disgust': 1704,
 'embarrassment': 817,
 'fear': 1048,
 'grief': 227,
 'nervousness': 598,
 'remorse': 849,
 'sadness': 2193,
 'confusion': 2471,
 'curiosity': 3267,
 'realization': 2867,
 'surprise': 1806,
 'neutral': 18423}

In [8]:
category_count

{'positive': 32798, 'negative': 21013, 'neutral': 28834}

In [9]:
# Sentiment analysis labels:
# 0 : Negative, 1 : Positive, 2: Neutral/Ambigous

col_names = ['text','sentiment_label']

new_data = []
for id,row in train_df.iterrows():
    if row['example_very_unclear'] == True:
        continue
    else:
        if row['neutral'] == True:
            data = [row['text'], 2]
        else:
            pos = 0
            neg = 0
            neut = 0
            max_cnt = -1
            for label in pos_labels:
                if row[label] == 1:
                    pos += 1
            max_cnt = max(pos,max_cnt)
            for label in neg_labels:
                if row[label] == 1:
                    neg += 1
            max_cnt = max(neg,max_cnt)
            for label in ambi_labels:
                if row[label] == 1:
                    neut += 1
            max_cnt = max(neut,max_cnt)
            if max_cnt == pos:
                data = [row['text'], 1]
            elif max_cnt == neg:
                data = [row['text'], 0]
            else:
                data = [row['text'], 2]
        new_data.append(data)
    
sentiment_train_data = pd.DataFrame(np.array(new_data),columns=col_names)
sentiment_train_data.head()

,text,sentiment_label
0,That game hurt.,0
1,"You do right, if you don't care then fuck 'em!",2
2,Man I love reddit.,1
3,"[NAME] was nowhere near them, he was by the Fa...",2
4,Right? Considering it’s such an important docu...,1


In [10]:
sentiment_train_data.to_csv(data_path + 'sentiment_train.csv')

In [11]:
anger_list = [ "anger", "annoyance", "disapproval", "disgust"]
fear_list = ["fear", "nervousness"]
joy_list = ["joy", "amusement", "approval", "excitement", "gratitude","love", "optimism", "relief", "pride", "admiration", "desire", "caring"]
sadness_list = ["sadness", "disappointment", "embarrassment", "grief", "remorse"]
surprise_list = ["surprise", "realization", "confusion", "curiosity"]

In [12]:
emotion_groups = [anger_list, fear_list, joy_list, sadness_list, surprise_list]

In [13]:
for ix, eg in enumerate(emotion_groups):
    print(ix)
    print(eg)
    print("*******")

0
['anger', 'annoyance', 'disapproval', 'disgust']
*******
1
['fear', 'nervousness']
*******
2
['joy', 'amusement', 'approval', 'excitement', 'gratitude', 'love', 'optimism', 'relief', 'pride', 'admiration', 'desire', 'caring']
*******
3
['sadness', 'disappointment', 'embarrassment', 'grief', 'remorse']
*******
4
['surprise', 'realization', 'confusion', 'curiosity']
*******


In [14]:
# Grouping emotions:

"""
Labels:
Anger (0) : [“Anger”, “annoyance”, “disapproval”, “disgust”]
Fear (1) : [“fear”, “nervousness” ]
Joy (2) : [“joy” , “amusement”, “approval”, “excitement”, “gratitude”,
     “love”, “optimism”, “relief”, “pride”, “admiration”, “desire”, “caring”]
Sadness (3) : [“Sadness”, “Disappointment”, “Embarrassment”, “grief”, “remorse”]
Surprise (4) : [“Surprise”, “Realization”, “confusion”, “curiosity”]
Neutral (5) : ["Neutral"]
"""

col_names = ['text','group_label']

new_data = []
for id,row in train_df.iterrows():
    if row['example_very_unclear'] == True:
        continue
    else:
        if row['neutral'] == True:
            data = [row['text'], 5]
        else:
            max_cnt = -1
            max_label = -1
            for ix,eg in enumerate(emotion_groups):
                cnt = 0
                for label in eg:
                    if row[label] == 1:
                        cnt += 1
                if cnt > max_cnt:
                    max_cnt = cnt
                    max_label = ix
            data = [row['text'], max_label]
        new_data.append(data)
    
emotion_group_train = pd.DataFrame(np.array(new_data),columns=col_names)
emotion_group_train.head()

,text,group_label
0,That game hurt.,3
1,"You do right, if you don't care then fuck 'em!",5
2,Man I love reddit.,2
3,"[NAME] was nowhere near them, he was by the Fa...",5
4,Right? Considering it’s such an important docu...,2


In [15]:
emotion_group_train.to_csv(data_path + 'group_train.csv')